In [12]:
import pandas as pd
import numpy as np 
import os, sys
sys.path.append('/home/jessica/labelGAN/StyleGAN.pytorch/')
from models.GAN import * 
from stylegan_layers import  G_mapping,G_synthesis
from sklearn.neighbors import KernelDensity
import shutil
from torchvision.utils import save_image
import torch

In [2]:
device = 'cuda:2' if torch.cuda.is_available() else 'cpu'
g_all = nn.Sequential(OrderedDict([('g_mapping', G_mapping()),
    #('truncation', Truncation(avg_latent)),
    ('g_synthesis', G_synthesis(resolution=1024))    
    ]))

opts = {'mapping_layers': 8, 'truncation_psi': -1.}
'''g_all = Generator(resolution=1024,
                    num_channels=3,
                    structure='linear',
                    **opts)'''
g_all = nn.Sequential(OrderedDict([('g_mapping', GMapping()),
    #('truncation', Truncation(avg_latent)),
    ('g_synthesis', GSynthesis(resolution=1024, depth=8))    
    ]))
#Load the pre-trained model
g_all.load_state_dict(torch.load('/data1/jessica/data/labelGAN/checkpoints/styleGAN/GAN_GEN_8.pth', map_location=device), strict=False)
g_all.eval()
g_all.to(device)
g_mapping, g_synthesis = g_all[0], g_all[1]

In [3]:
def get_sampls(kde):
    # Generate a set of points at which to evaluate the PDF
    x_grid = np.linspace(-3, 3, 1000)

    # Compute the estimated PDF at the evaluation points
    pdf_vals = np.exp(kde.score_samples(x_grid.reshape(-1, 1)))

    # Compute the CDF of the estimated PDF
    cdf_vals = np.cumsum(pdf_vals) / np.sum(pdf_vals)

    # Generate 100 new samples using inverse transform sampling
    u = np.random.uniform(size=100)
    new_samples = np.interp(u, cdf_vals, x_grid)
    return new_samples

In [6]:
image_latents_names = [elem.replace("latent_", "").replace(".npy", "") for elem in os.listdir('/home/jessica/labelGAN/Image2StyleGAN/images/latents')]
df = pd.read_csv("vinbig_25_samples.csv")
df_sampled = df[df.image_id.isin(image_latents_names)]
kernel = 'tophat'
for class_name in df.class_name.unique():
   print(class_name)
   test = df_sampled[df_sampled.class_name == class_name].image_id
   class_name = class_name.replace("/", "").replace(" ", "")
   images = test.apply(lambda x: np.load("./images/latents/latent_" + x + ".npy"))
   if len(images) == 0: continue
   t = np.concatenate(np.array(images), axis=0)
   t = t.reshape(t.shape[0], -1)# create an array of samples
   samples = np.array(t)

   # fit a kernel density estimator to the samples
   kde = KernelDensity(bandwidth=0.5, kernel=kernel)
   kde.fit(samples)

   # sample from the kde
   new_samples = kde.sample(100)
   for i, latents in enumerate(new_samples):
      latents = torch.tensor(latents.reshape(1, 18, 512)).to("cuda:2").float()
      np.save(f"./images/generated_latents_from_class_distr/{class_name}_{i}_{kernel}.npy", latents.cpu())
      syn_img = g_synthesis(latents, depth=8)
      syn_img = (syn_img+1.0)/2.0
      save_image(syn_img.clamp(0,1),f"./images/generated_images_from_class_distr/{class_name}_{i}_{kernel}.png")

No finding
Aortic enlargement
Pleural thickening
ILD
Nodule/Mass
Pulmonary fibrosis
Cardiomegaly
Pleural effusion
Calcification
Other lesion
Lung Opacity
Infiltration
Consolidation
Atelectasis
Pneumothorax


In [7]:
df.image_id == image_id

,Unnamed: 0,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
0,1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN
1,4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN
2,33,42d472bdda3ad93dac63c8e5e29977bb,No finding,14,R9,NaN,NaN,NaN,NaN
3,34,2b1293d9c276e5439e499f58ce2e31ab,No finding,14,R8,NaN,NaN,NaN,NaN
4,65,2561ba15f063a8e3a0322ad943738bb5,No finding,14,R2,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
370,14749,31ffa866e091965da25a408b4cc7d0db,Pneumothorax,12,R9,1212.0,355.0,1868.0,1192.0
371,15725,0f45742c4d100eeee221f8853d79c9d4,Pneumothorax,12,R10,308.0,1511.0,430.0,1669.0
372,15862,3c63e58fcda26e02fdd6619515399985,Pneumothorax,12,R9,226.0,340.0,950.0,1327.0
373,16769,393909c0d12d67fd08703d36970014e8,Pneumothorax,12,R10,1817.0,515.0,2249.0,1485.0


In [24]:
latents = []
p = '/home/jessica/labelGAN/Image2StyleGAN/images/generated_latents_from_class_distr/'
for i, elem in enumerate(os.listdir(p)):
    latents.append(np.load(p + elem ))
    
    

In [27]:
len(os.listdir(p))

1804

In [26]:
np.array(latents).shape

(1804, 1, 18, 512)

In [22]:
path = '/home/jessica/labelGAN/Image2StyleGAN/images/generated_latents_from_class_distr/Atelectasis_1_tophat.npy'
a = np.load(path)

In [23]:
a.shape

(1, 18, 512)